# LangChain Cohere Agent using Tools

## Install Libraries

In [1]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4
# ! pip install markdown
# ! pip install wikipedia
# ! pip install pyowm
# ! pip install yfinance

## Imports for Tool Use and Keys
### API Keys
* https://cohere.com/
* https://www.alphavantage.co/
* https://qdrant.tech/
* https://openweathermap.org/
* https://tavily.com/
* https://polygon.io/


In [1]:
from IPython.display import HTML
import markdown
import warnings
import streamlit as st
import os
from langchain_cohere.llms import Cohere
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain.agents import AgentExecutor
from cohere_tools import get_internet_search_tool, get_wikipedia_tool, get_openweathermap_tool
from yfinance_tools import get_business_summary_tool, get_ticker_history_tool, get_ticker_info_tool
from alpha_vantage_tools import (
    get_search_symbols_tool,
    get_exchange_rate_tool,
    # get_time_series_daily_tool,
    # get_time_series_weekly_tool,
    # get_market_news_sentimen_tool,
    # get_top_gainers_losers_tool,
)

warnings.filterwarnings("ignore")

ALPHAVANTAGE_API_KEY = st.secrets["ALPHAVANTAGE_API_KEY"]
os.environ["ALPHAVANTAGE_API_KEY"] = st.secrets["ALPHAVANTAGE_API_KEY"]

COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]

QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY

POLYGON_API_KEY = st.secrets["POLYGON_API_KEY"]
os.environ["POLYGON_API_KEY"] = st.secrets["POLYGON_API_KEY"]

TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

## Invoke ChatCohere

In [3]:
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

content="Who's there?" additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '507e08c2-3681-4fa5-8e12-b0203907215d', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '507e08c2-3681-4fa5-8e12-b0203907215d', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} id='run-caf585fa-de2d-4a32-99b4-49cd570f8130-0'


## Invoke ChatCohere using ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "bears"})

AIMessage(content='Why don’t bears wear socks?\n\nThey have bear feet!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'ce428c02-da5f-4a6b-bedf-86bd2b8f9f88', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'ce428c02-da5f-4a6b-bedf-86bd2b8f9f88', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, id='run-9f05895f-09c1-4475-914f-ddebd173e9ab-0')

## Invoke Cohere

In [5]:
llm = Cohere()
display(HTML(markdown.markdown(llm.invoke("Summarize the theory of Kahnemann and Tversky"))))

## Invoke Cohere with Tools

In [2]:
def invoke_cohere_with_tools(input, tools, prompt_template="{input}",no_markdown=False,verbose=False):
    llm = ChatCohere()
    prompt = ChatPromptTemplate.from_template(prompt_template)
    agent = create_cohere_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose)
    response = agent_executor.invoke(input)
    if no_markdown:
        print(response['output'])
    else:
        display(HTML(markdown.markdown(response['output'])))



## Tavily Internet Search Tool

In [7]:
prompt_template = "In which year had {stock_name} its IPO? Tell me the year and the date."
input =  {"stock_name": "Amazon"}
tools = [get_internet_search_tool()]
invoke_cohere_with_tools(input, tools, prompt_template)

## Wikipedia Tool

In [8]:
input =  {"input": "Explain the theory of Kahnemann and Tversky"}
tools = [get_wikipedia_tool()]
invoke_cohere_with_tools(input, tools)

## Run tool without agent

In [9]:
wikipedia_tool = get_wikipedia_tool()
wikipedia_tool.run("David Beckham")

"Page: David Beckham\nSummary: David Robert Joseph Beckham  ( BEK-əm; born 2 May 1975) is an English former professional footballer, the president and co-owner of Inter Miami CF and co-owner of Salford City. Primarily a right winger and known for his range of passing, crossing ability and free-kick taking, Beckham has been hailed as one of the greatest and most recognisable midfielders of his generation, as well as one of the best free-kick takers of all time. He won 19 major trophies in his career, and is the first English player to win league titles in four different countries: England, Spain, the United States and France.\nBeckham's professional club career began with Manchester United, where he made his first-team debut in 1992 at age 17. With United, he won the Premier League title six times, the FA Cup twice, the FA Charity Shield twice, the Intercontinental Cup and the UEFA Champions League in 1999. He then played four seasons with Real Madrid, winning the La Liga championship i

## Openweathermap Tool

In [10]:
input = {"input": "How is the weather in Salvador da Bahia?"}
tools = [get_openweathermap_tool()]
invoke_cohere_with_tools(input, tools)

## Multiple Tool Use
1. wiki-tool
2. open_weather_map
3. internet_search

In [ ]:
multiple_tools = [get_openweathermap_tool(), get_internet_search_tool(), get_wikipedia_tool()]
prompt_template = "Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}"

input = {"input": "Who was George Washington?"}
invoke_cohere_with_tools(input, multiple_tools, prompt_template)

input = {"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}
invoke_cohere_with_tools(input, multiple_tools, prompt_template)

input = {"input": "Whats in the news today?"}
invoke_cohere_with_tools(input, multiple_tools, prompt_template)

## Yfinance Tools

In [3]:
yfinance_tools = [get_business_summary_tool(),get_ticker_history_tool(), get_ticker_info_tool()]

### Business Summary Tool

In [13]:
prompt_template_summary = "Give me a business summary for {ticker}." 
input = {"ticker": "MSFT"}
invoke_cohere_with_tools(input=input,tools=yfinance_tools,prompt_template=prompt_template_summary)

### Ticker History Tool

In [14]:
prompt_template_history = "How did {ticker} perform the last {period}? Show the timeseries in a dataframe and calculate and rate the performance."
ticker = 'AMZN'
period = '5d'
input = {"ticker": {ticker}, "period": {period}}
invoke_cohere_with_tools(input=input, tools=yfinance_tools,prompt_template=prompt_template_history)

### Ticker Info Tool with Investment Rating

In [15]:
prompt_template_ticker_info = """I need detailed information about {ticker_symbol}. Produce a key value list for the 25 most important {topics} using CRLF before each point.
                                 After that analyze the fundamental and technical data and rate the company. Is it a good buy? Is it oversold or overbought? Is it a stable investment
                                 with a good sharpe ratio?"""
yfinance_tools = [get_ticker_info_tool()]
input = {"ticker_symbol": "AMSC", "topics": "key figures, technical indicators and analyst recommendations"}

invoke_cohere_with_tools(input=input, tools=yfinance_tools, prompt_template=prompt_template_ticker_info)

## Alpha Vantage Tool

In [ ]:
yfinance_tools = [
    # get_search_symbol_tool(),
    get_exchange_rate_tool(),
    # get_time_series_daily_tool(),
    # get_time_series_weekly_tool(),
    # get_market_news_sentimen_tool(),
    # get_top_gainers_losers_tool(),
]
# alpha_vantage = AlphaVantageAPIWrapper()
# alpha_vantage._get_exchange_rate("USD", "JPY")
# alpha_vantage._get_time_series_daily("MSFT")
# alpha_vantage._get_time_series_weekly("IBM")
# alpha_vantage.search_symbols("American Superconductor")
# alpha_vantage._get_market_news_sentiment("Microsoft")
# alpha_vantage._get_top_gainers_losers()
# response = agent_executor.invoke(
#     {"input": "What is the symbol for American Superconductor?"}
# )
input = {"input": "How much is one USD in EUR?"}
invoke_cohere_with_tools(input, yfinance_tools, no_markdown= True)
# response = agent_executor.invoke({"input": "Who are the gainers and losers at the stock exchange?"})
# response = agent_executor.invoke({"input": "How is Microsoft performing historically?"})
# response = agent_executor.invoke({"input": "How did Apple perform the last time?"})
# response = agent_executor.invoke({"input": "What is in the news about Nvidia?"})

## Polygon Toolkit

### Polygon Tools

In [4]:
polygon = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
polygon_tools = toolkit.get_tools()

In [5]:
for tool in polygon_tools:
    schema = str(tool.args_schema.schema())
    tool_documentation = f"<h1>{tool.name}</h1>\n\n<h2>Description</h2>{tool.description}\n\n<h2>Schema</h2>{schema}"
    display(HTML(markdown.markdown(tool_documentation)))

### Aggregates

In [ ]:
# Poloygon is called with the correct arguments but this produces an API error
# input = {"input": "How did MSFT perform the last 5 days?"}
# invoke_cohere_with_tools(tools=tools,input=input)

In [ ]:
from langchain_community.tools.polygon.aggregates import PolygonAggregates
from langchain_community.tools.polygon.financials import PolygonFinancials
from langchain_community.tools.polygon.last_quote import PolygonLastQuote
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools.polygon.aggregates import PolygonAggregatesSchema

api_wrapper = PolygonAPIWrapper()
ticker = "AAPL"

# Define param
params = PolygonAggregatesSchema(
    ticker=ticker,
    timespan="day",
    timespan_multiplier=1,
    from_date="2024-03-01",
    to_date="2024-03-08",
)
# Get aggregates for ticker
aggregates_tool = PolygonAggregates(api_wrapper=api_wrapper)
aggregates = aggregates_tool.run(tool_input=params.dict())
print(aggregates)

### Last Quote

In [ ]:
# Works only with premium subscription at Polygon
# input = {"input": "What is the end of day price for MSFT?"}
# invoke_cohere_with_tools(tools=polygon_tools,input=input)

### Ticker News Sentiment

In [ ]:
prompt_template_av_sentiment = "What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating."
ticker = "AMSC"
input = {"ticker": ticker}
invoke_cohere_with_tools(tools=polygon_tools,input=input,prompt_template=prompt_template_av_sentiment)

### Financials

In [ ]:
prompt_template_av_financials = "How are the financials for {ticker}? What is its market value? Produce an overview in form of a bullet point list (use CRLF for new lines) and rate the financial situation of the company."
ticker = "AMSC"
input = {"ticker": ticker}
invoke_cohere_with_tools(tools=polygon_tools,input=input,prompt_template=prompt_template_av_financials)

## Financials, News and Technical Combined

In [6]:
combined_tools = []
filtered_polygon_tools = [tool for tool in polygon_tools if tool.name in ("polygon_financials", "polygon_ticker_news")]

combined_tools = filtered_polygon_tools
for tool in yfinance_tools:
    combined_tools.append(tool)

prompt_template_av_financials = """ How are the financials for {ticker}? What is its market value?
                                    Produce an overview in form of a bullet point list (use CRLF for new lines)
                                    and rate the financial situation of the company. """


prompt_template_av_sentiment = " What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating. "

prompt_template_ticker_info = """ I need detailed information about {ticker}. Produce a key value list for the 25 most important {topics} using CRLF before each point.
                                 After that analyze the fundamental and technical data and rate the company. Is it a good buy? Is it oversold or overbought? Is it a stable investment
                                 with a good sharpe ratio? """

prompt_template_history = " How did {ticker} perform the last {period}? Show the timeseries in a dataframe and calculate and rate the performance. "

prompt_template_summary = " Give me a business summary for {ticker}. " 

prompt_template_rating = "Based on the information for {ticker} produce a profound investment rating and explain why the stock is a buy or not."

prompt_template_combined = prompt_template_summary + prompt_template_av_financials + prompt_template_av_sentiment + prompt_template_history + prompt_template_ticker_info + prompt_template_rating

ticker = "AMSC"
period = "1y"
topics = "key figures, technical indicators and analyst recommendations"
input = {"ticker": ticker, "topics": topics, "period": period}
invoke_cohere_with_tools(input=input,tools=combined_tools,prompt_template=prompt_template_combined)

## Imports for Embeddings and Qdrant

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
import qdrant_client

## Build Qdrant Vector Store

In [ ]:
# web_links = ["https://www.fielmann-group.com/ueber-uns/?",
#              "https://de.wikipedia.org/wiki/Fielmann",
#              "https://de.wikipedia.org/wiki/G%C3%BCnther_Fielmann",
#              "https://de.wikipedia.org/wiki/Marc_Fielmann"] 

web_links = ["https://python.langchain.com/v0.2/docs/introduction",
             "https://python.langchain.com/v0.2/docs/concepts",
             "https://python.langchain.com/v0.2/docs/integrations/tools",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores",
             "https://python.langchain.com/v0.2/docs/integrations/document_loaders",
             "https://python.langchain.com/v0.2/docs/integrations/retrievers",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores/qdrant",
             "https://python.langchain.com/v0.2/docs/integrations/providers/cohere/#react-agent",
             "https://de.wikipedia.org/wiki/Herz#:~:text=Das%20Herz%20(lateinisch%20Cor%2C%20griechisch,die%20Versorgung%20aller%20Organe%20sichert."]
 

loader = WebBaseLoader(web_links)
document=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(document)

embeddings = CohereEmbeddings(model = "embed-english-v2.0")

qdrant = Qdrant.from_documents(
    document,
    embeddings,
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="rag_documents",
    force_recreate=True,
)


## Load Qdrant Vector Store

In [ ]:
client = qdrant_client.QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)
embeddings = CohereEmbeddings(model="embed-english-v2.0")
vector_store = Qdrant(
    client = client,
    collection_name = "rag_documents",
    embeddings = embeddings
)
print("connection established !")

## Use Qdrant Vector Store as RAG Retriever for Cohere

In [ ]:
prompt_template = """
You are a bot to answer questions from a document.
    
You will be given a context of the conversation made so far followed by a question, 
give the answer to the question using the context. 
The answer should be short, straight and to the point. If you don't know the answer, reply that the answer is not available.

Context: {context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = { "prompt" : PROMPT }

llm = ChatCohere()

template = (
        """Combine the chat history and follow up question into 
        a standalone question. 
        If chat hsitory is empty, use the follow up question as it is.
        Chat History: {chat_history}
        Follow up question: {question}"""
    )

prompt = PromptTemplate.from_template(template)

chain = ConversationalRetrievalChain.from_llm(     
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = vector_store.as_retriever(search_type="mmr"),
    condense_question_prompt = prompt,
    return_source_documents=False,
    combine_docs_chain_kwargs=chain_type_kwargs,
)
customer_prompt = "What are tools in LangChain?"


llm_response = chain.invoke(
                {
                    "context": chain.memory.buffer,
                    "question": customer_prompt,
                },
                return_only_outputs=True,
            )

display(HTML(markdown.markdown(llm_response['answer'])))
